# **Mercado salarial**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "salarios" in nombre and 'hidalgo' in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_salarios_hidalgo']

# 1. Volumen del mercado salarial de parejas

In [2]:
# Seleccionar las columnas 'id' y 'categoria'
df = df_salarios_hidalgo[['Volumen', 'Pareja','Categoria_Pareja']]

In [7]:
df['categoria_pareja'].isna().sum()

20

In [4]:
# Crea un diccionario para almacenar los DataFrames de cada categoría única
dataframes_por_categoria = {}

# Itera sobre los grupos creados por groupby()
for categoria, grupo in df.groupby('categoria_pareja'):
    # Almacena el DataFrame del grupo en el diccionario
    dataframes_por_categoria[categoria] = grupo

# Itera sobre las claves del diccionario dataframes_por_categoria
for categoria in dataframes_por_categoria:
    # Imprime la categoría actual
    print("DataFrame para categoria_pareja '{}':".format(categoria))
    # Imprime el DataFrame correspondiente a la categoría actual
    print(dataframes_por_categoria[categoria])


DataFrame para Categoria_Pareja 'A':
        Volumen   Pareja Categoria_Pareja
283   2130000.0  50700.0                A
302   1590000.0  50700.0                A
303   2160000.0  50700.0                A
304   1110000.0  50700.0                A
305  19296000.0  54080.0                A
DataFrame para Categoria_Pareja 'A307':
      Volumen   Pareja Categoria_Pareja
306  945000.0  59150.0             A307
DataFrame para Categoria_Pareja 'B':
         Volumen    Pareja Categoria_Pareja
178   3173200.00  41896.16                B
296  12312000.00  40560.00                B
297  26442528.14  42078.98                B
298   1375000.00  42250.00                B
299  27976000.00  43940.00                B
300   1612000.00  43940.00                B
DataFrame para Categoria_Pareja 'B302':
        Volumen   Pareja Categoria_Pareja
301  37962000.0  45630.0             B302
DataFrame para Categoria_Pareja 'C':
          Volumen    Pareja Categoria_Pareja
144  1.279250e+07  32461.45             

In [15]:
sumas_volumen_por_categoria = {}

# Itera sobre el diccionario de DataFrames
for categoria, df in dataframes_por_categoria.items():
    suma_volumen = df['Volumen'].sum() # Calcula la suma del volumen para cada DataFrame    
    sumas_volumen_por_categoria[categoria[0]] = suma_volumen # Guarda la suma del volumen en el diccionario, asegurándose de que la categoría sea la primer letra

# Convierte el diccionario de sumas en un DataFrame
df_sumas_volumen = pd.DataFrame.from_dict(sumas_volumen_por_categoria, orient='index', columns=['Suma_Volumen'])
# Mantener orden
nuevo_orden = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S']
df_sumas_volumen = df_sumas_volumen.reindex(nuevo_orden)
df_sumas_volumen['Suma_Volumen'] = df_sumas_volumen['Suma_Volumen'].fillna(0)

#pd.set_option('display.float_format', lambda x: '%.2f' % x) # Mostar números sin notación científica
pd.options.display.float_format = '{:,.2f}'.format # Mostrar comas como separadores de miles y dos decimales

print("DataFrame con las sumas del volumen por categoría:")
df_sumas_volumen

DataFrame con las sumas del volumen por categoría:


,Suma_Volumen
G,"191,472,819.75"
F,"12,288,070.00"
F1,0.00
E,"12,235,600.00"
E1,0.00
D,"20,096,950.00"
D1,0.00
C,"16,304,650.00"
C1,0.00
B,"37,962,000.00"


In [16]:
df_volumen_ms_parejas = df_sumas_volumen[['Suma_Volumen']]
#df_volumen_ms_parejas['Label'] = ['<5000 G','<5-8K F','8-12K F1','12-16K E','16-20K E1','20-25K D','25-30K D1','30-35K C','35-40K C1','40-45K B','45-50K B1','50-55K A','55-60K A1','60K S+']
df_volumen_ms_parejas['Label'] = [
    'G [Menor a 5000 mil]',
    'F [5 mil-8 mil]',
    'F1 [8 mil-12mil]',
    'E [12 mil-16mil]',
    'E1 [16 mil-20 mil]',
    'D [20 mil -25 mil]',
    'D1 [25 mil-30mil]',
    'C [30 mil-35 mil]',
    'C1 [35 mil-40 mil]',
    'B [40 mil-45 mil]',
    'B1 [45 mil-50 mil]',
    'A [50 mil-55 mil]',
    'A1 [55 mil-60 mil]',
    'S+ [Mayor a 60 mil]'
]
df_volumen_ms_parejas

,Suma_Volumen,Label
G,"191,472,819.75",G [Menor a 5000 mil]
F,"12,288,070.00",F [5 mil-8 mil]
F1,0.00,F1 [8 mil-12mil]
E,"12,235,600.00",E [12 mil-16mil]
E1,0.00,E1 [16 mil-20 mil]
D,"20,096,950.00",D [20 mil -25 mil]
D1,0.00,D1 [25 mil-30mil]
C,"16,304,650.00",C [30 mil-35 mil]
C1,0.00,C1 [35 mil-40 mil]
B,"37,962,000.00",B [40 mil-45 mil]


In [17]:
pd.options.display.float_format = '{:,.2f}'.format # Mostrar comas como separadores de miles y dos decimales
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_volumen_ms_parejas['Label'],  # Eje x: Categoría
    y=df_volumen_ms_parejas['Suma_Volumen'],  # Eje y: Cantidad
    text=df_volumen_ms_parejas['Suma_Volumen'].apply(lambda x: '{:,.2f}'.format(x)),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Volumen del mercado salarial de parejas',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='Total valor',  # Título del eje y
        tickvals=[1000000000, 2000000000, 3000000000, 4000000000, 5000000000],  # Posiciones de las líneas del eje y
        ticktext=['$1,000,000,000', '$2,000,000,000', '$3,000,000,000', '$4,000,000,000', '$5,000,000,000'],  # Etiquetas de las líneas del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

fig.show()

In [6]:
pd.options.display.float_format = '{:,.2f}'.format # Mostrar comas como separadores de miles y dos decimales
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_volumen_ms_parejas['Label'],  # Eje x: Categoría
    y=df_volumen_ms_parejas['Suma_Volumen'],  # Eje y: Cantidad
    text=df_volumen_ms_parejas['Suma_Volumen'].apply(lambda x: '{:,.2f}'.format(x)),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Volumen del mercado salarial de parejas',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='Total valor',  # Título del eje y
        tickvals=[1000000000, 2000000000, 3000000000, 4000000000, 5000000000],  # Posiciones de las líneas del eje y
        ticktext=['$1,000,000,000', '$2,000,000,000', '$3,000,000,000', '$4,000,000,000', '$5,000,000,000'],  # Etiquetas de las líneas del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas_vol',  carpeta='assets/graficas')

fig.show()

### DIRECTOS

In [3]:
# Datos de ejemplo
data = {
    'Suma_Volumen': [
        282512281,4420667429,4828793682,3689430716,1892944139,1158774394,98925625,156854581,88975467,315722622,105259760,189516296,32223200,0
    ],
    'Label': [
        '<5000 G', '<5-8K F', '8-12K F1', '12-16K E', '16-20K E1', '20-25K D', '25-30K D1', '30-35K C', '35-40K C1', '40-45K B', '45-50K B1', '50-55K A', '55-60K A1', '60K S+'
    ]
    
}

# Crear el DataFrame
df_volumen_ms_parejas = pd.DataFrame(data)

In [4]:
pd.options.display.float_format = '{:,.2f}'.format # Mostrar comas como separadores de miles y dos decimales
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_volumen_ms_parejas['Label'],  # Eje x: Categoría
    y=df_volumen_ms_parejas['Suma_Volumen'],  # Eje y: Cantidad
    text=df_volumen_ms_parejas['Suma_Volumen'].apply(lambda x: '{:,.2f}'.format(x)),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Volumen del mercado salarial de parejas',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
       # title='Total valor',  # Título del eje y
        tickvals=[1000000000, 2000000000, 3000000000, 4000000000, 5000000000],  # Posiciones de las líneas del eje y
        ticktext=['$1,000,000,000', '$2,000,000,000', '$3,000,000,000', '$4,000,000,000', '$5,000,000,000'],  # Etiquetas de las líneas del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas_vol', carpeta='assets/graficas')

fig.show()


# 2. Mercado salario por pareja

In [7]:
df = df_salarios_tulancingo[['id_salarios','Volumen', 'Pareja','Categoria_Pareja']]

In [9]:
numero_registros_por_categoria = {}
# Itera dataframes_por_categoria que contiene cada grupo de categorías por pareja
for categoria in dataframes_por_categoria:
    numero_registros = len(dataframes_por_categoria[categoria]) # Número de registros para el DataFrame correspondiente a la categoría actual
    numero_registros_por_categoria[categoria] = numero_registros # Almacena en el diccionario 

# Convertir el diccionario en un DataFrame
posiciones_salariosparejas = pd.DataFrame(list(numero_registros_por_categoria.items()), columns=['Categoria_Pareja', 'numero_registros'])
# Mantener orden
nuevo_orden = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S']
posiciones_salariosparejas = posiciones_salariosparejas.set_index('Categoria_Pareja').reindex(nuevo_orden).fillna(0).reset_index()
posiciones_salariosparejas

,Categoria_Pareja,numero_registros
0,G,56.00
1,F,88.00
2,F1,96.00
3,E,36.00
4,E1,25.00
5,D,11.00
6,D1,9.00
7,C,3.00
8,C1,5.00
9,B,1.00


In [10]:
reemplazos = {
    'G': 'G [Menor a 5000 mil]',
    'F': 'F [5 mil-8 mil]',
    'F1': 'F1 [8 mil-12mil]',
    'E': 'E [12 mil-16mil]',
    'E1': 'E1 [16 mil-20 mil]',
    'D': 'D [20 mil -25 mil]',
    'D1': 'D1 [25 mil-30mil]',
    'C': 'C [30 mil-35 mil]',
    'C1': 'C1 [35 mil-40 mil]',
    'B': 'B [40 mil-45 mil]',
    'B1': 'B1 [45 mil-50 mil]',
    'A': 'A [50 mil-55 mil]',
    'A1': 'A1 [55 mil-60 mil]',
    'S': 'S+ [Mayor a 60 mil]'
}

# Reemplazar los valores en la columna 'Categoria_Pareja'
posiciones_salariosparejas['Categoria_Pareja'] = posiciones_salariosparejas['Categoria_Pareja'].replace(reemplazos)


,Categoria_Pareja,numero_registros
0,<5000 G,56.00
1,<5-8K F,88.00
2,8-12K F1,96.00
3,12-16K E,36.00
4,16-20K E1,25.00
5,20-25K D,11.00
6,25-30K D1,9.00
7,30-35K C,3.00
8,35-40K C1,5.00
9,40-45K B,1.00


In [11]:
total_registros = posiciones_salariosparejas['numero_registros'].sum() # Calcular el total de registros en el DataFrame
# Calculamos el porcentaje que representa cada numero_registros en relación al total de registros en el df
posiciones_salariosparejas['porcentaje'] = (posiciones_salariosparejas['numero_registros'] / total_registros) * 100
posiciones_salariosparejas

,Categoria_Pareja,numero_registros,porcentaje
0,<5000 G,56.00,16.82
1,<5-8K F,88.00,26.43
2,8-12K F1,96.00,28.83
3,12-16K E,36.00,10.81
4,16-20K E1,25.00,7.51
5,20-25K D,11.00,3.30
6,25-30K D1,9.00,2.70
7,30-35K C,3.00,0.90
8,35-40K C1,5.00,1.50
9,40-45K B,1.00,0.30


In [12]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=posiciones_salariosparejas['Categoria_Pareja'],  # Eje x: Categoría
    y=posiciones_salariosparejas['porcentaje'],  # Eje y: porcentaje
    text=posiciones_salariosparejas['porcentaje'].apply(lambda x: f'{x:.2f}%'),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Mercado salario por pareja',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='%',  # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    if not os.path.exists(carpeta): 
        os.makedirs(carpeta) # Crear la carpeta si no existe
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas',  carpeta='assets/graficas')

fig.show()

### DIRECTOS

In [5]:
# Datos de ejemplo
data = {
    'porcentaje': [
        9.23,14.46,24.00,18.46,11.69,8.62,2.77,2.77,1.23,2.46,0.00,1.23,0.62,0.00
    ],
    'Categoria_Pareja': [
        '<5000 G', '<5-8K F', '8-12K F1', '12-16K E', '16-20K E1', '20-25K D', '25-30K D1', '30-35K C', '35-40K C1', '40-45K B', '45-50K B1', '50-55K A', '55-60K A1', '60K S+'
    ]
    
}

# Crear el DataFrame
posiciones_salariosparejas = pd.DataFrame(data)

In [6]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=posiciones_salariosparejas['Categoria_Pareja'],  # Eje x: Categoría
    y=posiciones_salariosparejas['porcentaje'],  # Eje y: porcentaje
    text=posiciones_salariosparejas['porcentaje'].apply(lambda x: f'{x:.2f}%'),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Mercado salario por pareja',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='%',  # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    if not os.path.exists(carpeta): 
        os.makedirs(carpeta) # Crear la carpeta si no existe
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas',  carpeta='assets/graficas')

fig.show()

# 3. Mercado salario por persona

In [13]:
df = df_salarios_tulancingo[['id_salarios','Salario_Variable','Categoria']]

In [14]:
# Crea un diccionario para almacenar los DataFrames de cada categoría única
dataframes_por_categoria = {}

# Itera sobre los grupos creados por groupby()
for categoria, grupo in df.groupby('Categoria'):
    # Almacena el DataFrame del grupo en el diccionario
    dataframes_por_categoria[categoria] = grupo

# Itera sobre las claves del diccionario dataframes_por_categoria
for categoria in dataframes_por_categoria:
    # Imprime la categoría actual
    print("DataFrame para Categoria '{}':".format(categoria))
    # Imprime el DataFrame correspondiente a la categoría actual
    print(dataframes_por_categoria[categoria])


DataFrame para Categoria 'C':
     id_salarios  Salario_Variable Categoria
330          331         30,000.00         C
331          332         34,000.00         C
332          333         34,564.25         C
DataFrame para Categoria 'D':
     id_salarios  Salario_Variable Categoria
323          324         20,000.00         D
324          325         21,500.00         D
325          326         21,559.79         D
326          327         22,119.49         D
327          328         22,295.80         D
328          329         22,465.12         D
DataFrame para Categoria 'D1':
     id_salarios  Salario_Variable Categoria
329          330         25,800.00        D1
DataFrame para Categoria 'E':
     id_salarios  Salario_Variable Categoria
302          303         12,000.00         E
303          304         12,000.00         E
304          305         12,384.35         E
305          306         12,447.85         E
306          307         12,845.16         E
307          308        

In [15]:
numero_registros_por_categoria = {}
# Itera dataframes_por_categoria que contiene cada grupo de categorías por pareja
for categoria in dataframes_por_categoria:
    numero_registros = len(dataframes_por_categoria[categoria]) # Número de registros para el DataFrame correspondiente a la categoría actual
    numero_registros_por_categoria[categoria] = numero_registros # Almacena en el diccionario 

# Convertir el diccionario en un DataFrame
posiciones_salariospersona = pd.DataFrame(list(numero_registros_por_categoria.items()), columns=['Categoria', 'numero_registros'])
# Mantener orden
nuevo_orden = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S']
posiciones_salariospersona = posiciones_salariospersona.set_index('Categoria').reindex(nuevo_orden).fillna(0).reset_index()
posiciones_salariospersona

,Categoria,numero_registros
0,G,155.00
1,F,99.00
2,F1,48.00
3,E,14.00
4,E1,7.00
5,D,6.00
6,D1,1.00
7,C,3.00
8,C1,0.00
9,B,0.00


In [16]:
reemplazos = {
    'G': 'G [Menor a 5000 mil]',
    'F': 'F [5 mil-8 mil]',
    'F1': 'F1 [8 mil-12mil]',
    'E': 'E [12 mil-16mil]',
    'E1': 'E1 [16 mil-20 mil]',
    'D': 'D [20 mil -25 mil]',
    'D1': 'D1 [25 mil-30mil]',
    'C': 'C [30 mil-35 mil]',
    'C1': 'C1 [35 mil-40 mil]',
    'B': 'B [40 mil-45 mil]',
    'B1': 'B1 [45 mil-50 mil]',
    'A': 'A [50 mil-55 mil]',
    'A1': 'A1 [55 mil-60 mil]',
    'S': 'S+ [Mayor a 60 mil]'
}

# Reemplazar los valores en la columna 'Categoria_Pareja'
posiciones_salariospersona['Categoria'] = posiciones_salariospersona['Categoria'].replace(reemplazos)


,Categoria,numero_registros,porcentaje
0,<5000 G,155.00,46.55
1,<5-8K F,99.00,29.73
2,8-12K F1,48.00,14.41
3,12-16K E,14.00,4.20
4,16-20K E1,7.00,2.10
5,20-25K D,6.00,1.80
6,25-30K D1,1.00,0.30
7,30-35K C,3.00,0.90
8,35-40K C1,0.00,0.00
9,40-45K B,0.00,0.00


In [ ]:
#posiciones_salariospersona['Categoria'] = ['<5000 G', '<5-8K F', '8-12K F1', '12-16K E', '16-20K E1', '20-25K D', '25-30K D1', '30-35K C', '35-40K C1', '40-45K B', '45-50K B1', '50-55K A', '55-60K A1', '60K S+']
total_registros = posiciones_salariospersona['numero_registros'].sum() # Calcular el total de registros en el DataFrame
# Calculamos el porcentaje que representa cada numero_registros en relación al total de registros en el df
posiciones_salariospersona['porcentaje'] = (posiciones_salariospersona['numero_registros'] / total_registros) * 100
posiciones_salariospersona

In [17]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=posiciones_salariospersona['Categoria'],  # Eje x: Categoría
    y=posiciones_salariospersona['porcentaje'],  # Eje y: porcentaje
    text=posiciones_salariospersona['porcentaje'].apply(lambda x: f'{x:.2f}%'),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Mercado Salario por persona',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta): 
        os.makedirs(carpeta) # Crear la carpeta si no existe
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_persona', carpeta='assets/graficas')

fig.show()

### DIRECTOS

In [7]:
# Datos de ejemplo
data = {
    'porcentaje': [
        23.32,30.03,25.56,8.63,6.07,1.28,1.92,1.28,0.32,0.64,0.32,0.0,0.0,0.64
    ],
    'Categoria': [
        '<5000 G', '<5-8K F', '8-12K F1', '12-16K E', '16-20K E1', '20-25K D', '25-30K D1', '30-35K C', '35-40K C1', '40-45K B', '45-50K B1', '50-55K A', '55-60K A1', '60K S+'
    ]
    
}

# Crear el DataFrame
posiciones_salariospersona = pd.DataFrame(data)

In [8]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=posiciones_salariospersona['Categoria'],  # Eje x: Categoría
    y=posiciones_salariospersona['porcentaje'],  # Eje y: porcentaje
    text=posiciones_salariospersona['porcentaje'].apply(lambda x: f'{x:.2f}%'),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Mercado Salario por persona',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta): 
        os.makedirs(carpeta) # Crear la carpeta si no existe
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_persona', carpeta='assets/graficas')

fig.show()

# Por CSV

In [3]:
df = pd.read_csv('../../db/Scrapining/salarios/clean/TULANCINGO_24.csv')
df.columns

Index(['Occupation', 'Workforce', 'Monthly Wage', 'lastPeriod', 'diferencia',
       'diferencia_2', 'Salario_Variables', 'volumen', 'pareja',
       'categoria_persona', 'categoria_pareja'],
      dtype='object')

### 1. Volumen del mercado salarial de parejas

In [22]:
df = pd.read_csv('../../db/Scrapining/salarios/clean/TULANCINGO_24.csv')
categorias = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S+']
df['categoria_pareja'] = df['categoria_pareja'].replace('S', 'S+')
df['categoria_pareja'] = df['categoria_pareja'].apply(lambda x: x if x in categorias else 'Unknown')

# Ensure all categories are present, even if they have a sum of 0
categorias_completas = pd.DataFrame(categorias, columns=['categoria_pareja'])
df_sumas_volumen = df.groupby('categoria_pareja')['volumen'].sum().reset_index()
df_sumas_volumen.columns = ['categoria_pareja', 'suma_volumen']
df_sumas_volumen = pd.merge(categorias_completas, df_sumas_volumen, on='categoria_pareja', how='left').fillna(0)
df_sumas_volumen['suma_volumen'] = df_sumas_volumen['suma_volumen'].astype(float)
df_sumas_volumen

,categoria_pareja,suma_volumen
0,G,"261,357,813.87"
1,F,"1,690,903,073.05"
2,F1,"2,737,090,757.63"
3,E,"1,838,967,045.48"
4,E1,"1,011,504,001.12"
5,D,"467,123,926.67"
6,D1,"433,832,089.79"
7,C,"248,559,736.38"
8,C1,"98,331,987.03"
9,B,"44,451,409.51"


In [23]:
df_volumen_ms_parejas = df_sumas_volumen[['suma_volumen']]
labels = [
    'G [Menor a 5000 mil]',
    'F [5 mil-8 mil]',
    'F1 [8 mil-12mil]',
    'E [12 mil-16mil]',
    'E1 [16 mil-20 mil]',
    'D [20 mil -25 mil]',
    'D1 [25 mil-30mil]',
    'C [30 mil-35 mil]',
    'C1 [35 mil-40 mil]',
    'B [40 mil-45 mil]',
    'B1 [45 mil-50 mil]',
    'A [50 mil-55 mil]',
    'A1 [55 mil-60 mil]',
    'S+ [Mayor a 60 mil]'
]
categorias = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S+']
df_volumen_ms_parejas['categoria_pareja'] = categorias
df_volumen_ms_parejas['suma_volumen'] = df_volumen_ms_parejas.apply(lambda row: row['suma_volumen'] if row['categoria_pareja'] in df_sumas_volumen['categoria_pareja'].values else 0, axis=1)
df_volumen_ms_parejas['Label'] = labels
df_volumen_ms_parejas

,suma_volumen,categoria_pareja,Label
0,"261,357,813.87",G,G [Menor a 5000 mil]
1,"1,690,903,073.05",F,F [5 mil-8 mil]
2,"2,737,090,757.63",F1,F1 [8 mil-12mil]
3,"1,838,967,045.48",E,E [12 mil-16mil]
4,"1,011,504,001.12",E1,E1 [16 mil-20 mil]
5,"467,123,926.67",D,D [20 mil -25 mil]
6,"433,832,089.79",D1,D1 [25 mil-30mil]
7,"248,559,736.38",C,C [30 mil-35 mil]
8,"98,331,987.03",C1,C1 [35 mil-40 mil]
9,"44,451,409.51",B,B [40 mil-45 mil]


In [25]:
pd.options.display.float_format = '{:,.2f}'.format # Mostrar comas como separadores de miles y dos decimales
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_volumen_ms_parejas['Label'],  # Eje x: Categoría
    y=df_volumen_ms_parejas['suma_volumen'],  # Eje y: Cantidad
    text=df_volumen_ms_parejas['suma_volumen'].apply(lambda x: '{:,.2f}'.format(x)),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Volumen del mercado salarial de parejas',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='Total valor',  # Título del eje y
        tickvals=[1000000000, 2000000000, 3000000000, 4000000000, 5000000000],  # Posiciones de las líneas del eje y
        ticktext=['$1,000,000,000', '$2,000,000,000', '$3,000,000,000', '$4,000,000,000', '$5,000,000,000'],  # Etiquetas de las líneas del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas_vol', carpeta='assets/graficas')

fig.show()

### 2. Mercado salarial de parejas

In [15]:
# Organizar variable de categorías
categorias = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S+']
df['categoria_pareja'] = df['categoria_pareja'].str.upper()
df['categoria_pareja'] = df['categoria_pareja'].str.upper().replace('S', 'S+')
df['categoria_pareja'] = df['categoria_pareja'].apply(lambda x: x if x in categorias else 'Unknown') #Mantener orden

In [19]:
registros = df.groupby('categoria_pareja').size().reset_index(name='numero_registros')
registros = registros[registros['categoria_pareja'].isin(categorias)]
categorias = ['G', 'F', 'F1', 'E', 'E1', 'D', 'D1', 'C', 'C1', 'B', 'B1', 'A', 'A1', 'S+']

registros['orden'] = registros['categoria_pareja'].apply(lambda x: categorias.index(x))
registros = registros.sort_values(by='orden').drop('orden', axis=1)

registros

,categoria_pareja,numero_registros
11,G,21
9,F,38
10,F1,68
7,E,58
8,E1,40
5,D,27
6,D1,23
3,C,8
4,C1,7
1,B,6


In [ ]:
totales = 

In [ ]:
# Calculamos el porcentaje que representa cada numero_registros en relación al total de registros en el df
posiciones_salariosparejas['porcentaje'] = (posiciones_salariosparejas['numero_registros'] / total_registros) * 100
posiciones_salariosparejas

In [ ]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=posiciones_salariosparejas['Categoria_Pareja'],  # Eje x: Categoría
    y=posiciones_salariosparejas['porcentaje'],  # Eje y: porcentaje
    text=posiciones_salariosparejas['porcentaje'].apply(lambda x: f'{x:.2f}%'),  # Texto con el formato deseado
    textposition='outside',  # Posición del texto (parte superior de las barras)
    marker_color=colores,  # Especifica los colores de las barras
))
fig.update_layout(
    #title='Mercado salario por pareja',  # Título de la gráfica
    title_x=0.5,
    yaxis=dict(
        #title='%',  # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1, # Ancho de las líneas que dividen los rangos del eje Y
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    if not os.path.exists(carpeta): 
        os.makedirs(carpeta) # Crear la carpeta si no existe
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_mercadosalario_parejas',  carpeta='assets/graficas')

fig.show()

### 3. Mercado salarial de personas

In [ ]:
# Crea un diccionario para almacenar los DataFrames de cada categoría única
dataframes_por_categoria = {}

# Itera sobre los grupos creados por groupby()
for categoria, grupo in df.groupby('Categoria'):
    # Almacena el DataFrame del grupo en el diccionario
    dataframes_por_categoria[categoria] = grupo

# Itera sobre las claves del diccionario dataframes_por_categoria
for categoria in dataframes_por_categoria:
    # Imprime la categoría actual
    print("DataFrame para Categoria '{}':".format(categoria))
    # Imprime el DataFrame correspondiente a la categoría actual
    print(dataframes_por_categoria[categoria])
